# Tagging - Interpretando dados com funções

Uma das principais aplicações das funções externas por desenvolvedores não é exatamente na utilização dessas funções e sim, na utilização desta sintaxe e estruturação de dados gerado pela API da OpenAI para a categorização e estruturação de dados em texto. Daremos um exemplo aqui ao categorizanto sentimentos de falas.

In [1]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.utils.function_calling import convert_to_openai_function

class Sentimento(BaseModel):
    '''Define o sentimento e a língua da mensagem enviada'''
    sentimento: str = Field(description='Sentimendo do texto. Deve ser "pos", "neg" ou "nd" para não definido.')
    lingua: str = Field(description='Língua que o texto foi escrito (deve estar no formato ISO 639-1)')

tool_sentimento = convert_to_openai_function(Sentimento)
tool_sentimento

{'name': 'Sentimento',
 'description': 'Define o sentimento e a língua da mensagem enviada',
 'parameters': {'type': 'object',
  'properties': {'sentimento': {'description': 'Sentimendo do texto. Deve ser "pos", "neg" ou "nd" para não definido.',
    'type': 'string'},
   'lingua': {'description': 'Língua que o texto foi escrito (deve estar no formato ISO 639-1)',
    'type': 'string'}},
  'required': ['sentimento', 'lingua']}}

In [ ]:
texto = 'Eu gosto muito de massa aos quatro queijos'

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Pense com cuidado ao categorizar o texto conforme as instruções'),
    ('user', '{input}')
])
chat = ChatOpenAI()

chain = prompt | chat.bind(functions=[tool_sentimento], function_call={'name': 'Sentimento'})

In [6]:
chain.invoke({'input': 'Eu gosto muito de massa aos quatro queijos'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"pos","lingua":"pt"}', 'name': 'Sentimento'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 145, 'total_tokens': 156}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e68e3dfc-105c-4cc9-9a2a-f08e9c359f67-0')

In [7]:
chain.invoke({'input': 'I dont like this food'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"neg","lingua":"en"}', 'name': 'Sentimento'}}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 138, 'total_tokens': 149}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-101f6bd8-b4b9-4e49-b076-f18619b06d14-0')

## Parseando a saída para obtermos apenas o que interessa

Podemos utilizar o JsonOutputFunctionsParser para obtermos como resultado final da nossa chain apenas o que nos interessa, que é o dicionário com as tags do conteúdo.

In [8]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


chain = (prompt 
         | chat.bind(functions=[tool_sentimento], function_call={'name': 'Sentimento'})
         | JsonOutputFunctionsParser())
chain.invoke({'input': 'Eu gosto muito de massa aos quatro queijos'})

{'sentimento': 'pos', 'lingua': 'pt'}

In [9]:
chain.invoke({'input': 'I dont like this food'})

{'sentimento': 'neg', 'lingua': 'en'}

## Um exemplo mais interessante

Indo para um exemplo um pouco mais complexo. Digamos que temos um chatbot e queremos fazer um roteamento para os setores de interesse, no nosso caso atendimento_cliente, duvidas_alunos, vendas, spam. A primeira etapa é criar um modelo que entenda a solicitação do cliente e direcione para o setor certo. Nas próximas etapas o atendimento pode ser continuado por pessoas ou por agentes especializados para as tarefas do setor.
Retiramos as seguintes mensagens de email recebidos pela nossa equipe de atendimento e vamos tentar criar um direcionamento para elas:

In [14]:
duvidas = [
    'Bom dia, gostaria de saber se há um certificado final para cada trilha ou se os certificados são somente para os cursos e projetos? Obrigado!',
    'In Etsy, Amazon, eBay, Shopify https://pint77.com Pinterest+SEO +II = high sales results',
    'Boa tarde, estou iniciando hoje e estou perdido. Tenho vários objetivos. Não sei nada programação, exceto que utilizo o Power automate desktop da Microsoft. Quero aprender tudo na plataforma que se relacione ao Trading de criptomoedas. Quero automatizar Tradings, fazer o sistema reconhecer padrões, comprar e vender segundo critérios que eu defina, etc. Também tenho objetivos de aprender o máximo para utilizar em automações no trabalho também, que envolve a área jurídica e trabalho em processos. Como sou fã de eletrônica e tenho cursos na área, também queria aprender o que precisa para automatizacões diversas. Existe algum curso ou trilha que me prepare com base para todas essas áreas ao mesmo tempo e a partir dele eu aprenda isoladamente aquilo que seria exigido para aplicar aos meus projetos?',
    'Bom dia, Havia pedido cancelamento de minha mensalidade no mes 2 e continuaram cobrando. Peço cancelamento da assinatura. Peço por gentileza, para efetivarem o cancelamento da assomatura e pagamento.',
    'Bom dia. Não estou conseguindo tirar os certificados dos cursos que concluí. Por exemplo, já consegui 100% no python starter, porém, não consigo tirar o certificado. Como faço?',
    'Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?'
    ]

In [15]:
from enum import Enum
from langchain.pydantic_v1 import BaseModel, Field

class SetorEnum(str, Enum):
    atendimento_cliente = 'atendimento_cliente'
    duvidas_aluno = 'duvidas_aluno'
    vendas = 'vendas'
    spam = 'spam'

class DirecionaSetorResponsavel(BaseModel):
    """Direciona a dúvida de um cliente ou aluno da escola de programação Asimov para o setor responsável"""
    setor: SetorEnum

In [16]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_direcionamento = convert_to_openai_function(DirecionaSetorResponsavel)
tool_direcionamento

{'name': 'DirecionaSetorResponsavel',
 'description': 'Direciona a dúvida de um cliente ou aluno da escola de programação Asimov para o setor responsável',
 'parameters': {'type': 'object',
  'properties': {'setor': {'description': 'An enumeration.',
    'enum': ['atendimento_cliente', 'duvidas_aluno', 'vendas', 'spam'],
    'type': 'string'}},
  'required': ['setor']}}

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Pense com cuidado ao categorizar o texto conforme as instruções'),
    ('user', '{input}')
])
chat = ChatOpenAI()

chain = (prompt 
         | chat.bind(functions=[tool_direcionamento], function_call={'name': 'DirecionaSetorResponsavel'})
         | JsonOutputFunctionsParser())

In [23]:
duvida = duvidas[5]
resposta = chain.invoke({'input': duvida})
print('Dúvida:', duvida)
print('Resposta:', resposta)

Dúvida: Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?
Resposta: {'setor': 'atendimento_cliente'}


Neste caso, gostaríamos que a dúvida fosse direcionada para vendas. Podemos melhorar nosso prompt para auemntar a chance do modelo responder como gostaríamos:

In [24]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a dúvidas de preço, sobre o produto, como funciona devem ser direciodas para "vendas".
Questões relacionadas a conta, acesso a plataforma, a cancelamento e renovação de assinatura para devem ser direciodas para "atendimento_cliente".
Questões relacionadas a dúvidas técnicas de programação, conteúdos da plataforma ou tecnologias na área da programação devem ser direciodas para "duvidas_alunos".
Mensagens suspeitas, em outras línguas que não português, contendo links devem ser direciodas para "spam".
'''

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    ('user', '{input}')
])
chat = ChatOpenAI()

chain = (prompt 
         | chat.bind(functions=[tool_direcionamento], function_call={'name': 'DirecionaSetorResponsavel'})
         | JsonOutputFunctionsParser())

In [25]:
duvida = duvidas[5]
resposta = chain.invoke({'input': duvida})
print('Dúvida:', duvida)
print('Resposta:', resposta)

Dúvida: Bom dia. Não enconte no site o preço de um curso avulso. SAberiam me informar?
Resposta: {'setor': 'vendas'}
